In [ ]:
from tqdm import tqdm
import pandas as pd

import numpy as np
def round_geometry(g, decimals=1):
    try:
        coords = g.coords
    except NotImplementedError:
        return g
    
    coords = [(np.round(c[0], decimals), np.round(c[1], decimals)) for c in coords]
    return type(g)(coords)

def compress_geometry(g, decimals=1):
    return round_geometry(g.simplify(decimals))

def compress_geometries(self):

    projected_model = self.copy()
    iterator = tqdm(
        projected_model.__dict__.items(),
        desc='geometries'
    )

    for key, attribute in iterator:
        if isinstance(attribute, (gpd.GeoDataFrame, gpd.GeoSeries)):
            attribute['geometry'] = attribute['geometry'].apply(compress_geometry)
            projected_model.__setattr__(key, attribute)
        elif isinstance(attribute, pd.DataFrame):
            if 'geometry' in attribute.columns:
                # print('Converting {}'.format(key))
                attribute['geometry'] = attribute['geometry'].apply(compress_geometry)
                projected_model.__setattr__(key, attribute)
        elif isinstance(attribute, pd.Series):
            try:
                attribute['geometry'] = attribute['geometry'].apply(compress_geometry)
                projected_model.__setattr__(key, attribute)
            except:
                pass

    return projected_model